In [156]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [157]:
seasons = ['Borneo', 'The Australian Outback', 'Africa', 'Marquesas', 'Thailand', 'The Amazon', 
           'Pearl Islands', 'All-Stars', 'Vanuatu', 'Palau', 'Guatemala', 'Panama', 
           'Cook Islands', 'Fiji', 'China', 'Micronesia', 'Gabon', 'Tocantins', 'Samoa', 
           'Heroes vs. Villains', 'Nicaragua', 'Redemption Island', 'South Pacific',
           'One World', 'Philippines', 'Caramoan', 'Blood vs. Water', 'Cagayan', 'San Juan del Sur',
           'Worlds Apart', 'Cambodia', 'Kaôh Rōng', 'Millennials vs. Gen X',
           'Game Changers', 'Heroes vs. Healers vs. Hustlers', 'Ghost Island','David vs. Goliath']

west = ['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM']
midwest = ['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH']
south = ['OK', 'TX', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'WV', 'DC', 'MD', 'DE', 'VA', 'NC', 'SC', 'GA', 'FL']
northeast = ['PA', 'NY', 'VT', 'NH', 'ME', 'MA', 'CT', 'RI', 'NJ']

eastcoast = ['ME', 'NH', 'MA', 'RI', 'CT', 'NY', 'NJ', 'DE', 'ML', 'VA', 'NC', 'SC', 'GA', 'FL']
westcoast = ['CA', 'OR', 'WA']


In [158]:
ca = pd.read_csv('castaways.csv')

def createid(row):
    return str(row['ID']).zfill(4)

ca['ID'] = ca.apply(createid,axis=1)
ca.set_index('ID',inplace=True)
ca.sort_values(['season_number','place'], ascending=[True, False],inplace=True)
ca

,name,age,gender,state,season,place,days_survived,votes_against,place_percent,time_playing,total_times_played,players_in_season,season_number,is_censored
ID,,,,,,,,,,,,,,
0116,Sonja Christopher,63,Female,CA,Borneo,16,3,4,0.0625,1,1,16,1,0
0115,B.B. Andersen,64,Male,KS,Borneo,15,6,6,0.1250,1,1,16,1,0
0114,Stacey Stillman,27,Female,CA,Borneo,14,9,6,0.1875,1,1,16,1,0
0113,Ramona Gray,29,Female,NJ,Borneo,13,12,6,0.2500,1,1,16,1,0
0112,Dirk Been,23,Male,WI,Borneo,12,15,4,0.3125,1,1,16,1,0
0111,Joel Klug,27,Male,AR,Borneo,11,18,4,0.3750,1,1,16,1,0
0110,Gretchen Cordy,38,Female,TN,Borneo,10,21,4,0.4375,1,1,16,1,0
0109,Greg Buis,24,Male,CO,Borneo,9,24,6,0.5000,1,1,16,1,0
0108,Jenna Lewis,22,Female,NH,Borneo,8,27,11,0.5625,1,2,16,1,0


In [159]:
def map_state_to_region(state):
    if state in west:
        return 'West'
    elif state in midwest:
        return 'Midwest'
    elif state in south:
        return 'South'
    elif state in northeast:
        return 'Northeast'
    else:
        return 'NULL'
    
def map_state_to_coast(state):
    if state in eastcoast:
        return 'East Coast'
    elif state in westcoast:
        return 'West Coast'
    else:
        return 'Not Coast'
    
def map_season_to_school(season):
    if season in seasons[:18]:
        return 'Old School'
    else:
        return 'New School'
    
def map_age_group(age):
    if age <= 35:
        return 'Young'
    elif age <= 50:
        return 'Middle'
    else:
        return 'Old'

In [160]:
ca['region'] = ca['state'].map(map_state_to_region)
ca['coast'] = ca['state'].map(map_state_to_coast)
ca['school'] = ca['season'].map(map_season_to_school)
ca['agegroup'] = ca['age'].map(map_age_group)



In [161]:
ca['region'].value_counts()

West         252
South        200
Northeast    143
Midwest       78
Name: region, dtype: int64

In [162]:
test = ca.groupby(['region'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

region
Midwest      0.569770
Northeast    0.544716
South        0.498559
West         0.527338
Name: place_percent, dtype: float64

In [163]:
ca['coast'].value_counts()

Not Coast     249
East Coast    215
West Coast    209
Name: coast, dtype: int64

In [164]:
test = ca.groupby(['coast'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

coast
East Coast    0.527790
Not Coast     0.533782
West Coast    0.519383
Name: place_percent, dtype: float64

In [165]:
ca['school'].value_counts()

New School    362
Old School    311
Name: school, dtype: int64

In [166]:
test = ca.groupby(['school'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

school
New School    0.526243
Old School    0.528738
Name: place_percent, dtype: float64

In [167]:
ca['agegroup'].value_counts()

Young     445
Middle    177
Old        51
Name: agegroup, dtype: int64

In [168]:
test = ca.groupby(['agegroup'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

agegroup
Middle    0.518908
Old       0.526759
Young     0.530845
Name: place_percent, dtype: float64

In [169]:
ca['gender'].value_counts()

Male      338
Female    335
Name: gender, dtype: int64

In [170]:
treatments = []
test = ca.groupby(['gender'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

gender
Female    0.509743
Male      0.544893
Name: place_percent, dtype: float64

In [171]:
test = ca.groupby(['gender','agegroup'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

gender  agegroup
Female  Middle      0.508553
        Old         0.493726
        Young       0.511434
Male    Middle      0.528694
        Old         0.544777
        Young       0.551799
Name: place_percent, dtype: float64

In [172]:
test = ca.groupby(['gender','region'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

gender  region   
Female  Midwest      0.553437
        Northeast    0.535050
        South        0.482048
        West         0.501754
Male    Midwest      0.588824
        Northeast    0.554519
        South        0.513800
        West         0.552519
Name: place_percent, dtype: float64

In [173]:
test = ca.groupby(['gender','school'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

gender  school    
Female  New School    0.499753
        Old School    0.521344
Male    New School    0.552442
        Old School    0.536085
Name: place_percent, dtype: float64

In [174]:
test = ca.groupby(['agegroup','school'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

agegroup  school    
Middle    New School    0.514542
          Old School    0.524845
Old       New School    0.575000
          Old School    0.476588
Young     New School    0.525926
          Old School    0.536301
Name: place_percent, dtype: float64

In [175]:
test = ca.groupby(['agegroup','region'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

agegroup  region   
Middle    Midwest      0.590931
          Northeast    0.502691
          South        0.473512
          West         0.533563
Old       Midwest      0.424342
          Northeast    0.573214
          South        0.640764
          West         0.417328
Young     Midwest      0.562862
          Northeast    0.565372
          South        0.486414
          West         0.538227
Name: place_percent, dtype: float64

In [176]:
test = ca.groupby(['region','school'])['place_percent'].mean()
for x in range(len(test)):
    treatments.append([test.index[x],test[x]])
test

region     school    
Midwest    New School    0.566190
           Old School    0.572683
Northeast  New School    0.509357
           Old School    0.584826
South      New School    0.515942
           Old School    0.475041
West       New School    0.534109
           Old School    0.519399
Name: place_percent, dtype: float64

In [177]:
dtrt = pd.DataFrame(treatments)
dtrt.rename(index=str, columns={0: "Treatment", 1: "Placement_Percentage"},inplace=True)
dtrt.sort_values(by=['Placement_Percentage'],ascending=False,inplace=True)
dtrt

,Treatment,Placement_Percentage
32,"(Old, South)",0.640764
26,"(Middle, Midwest)",0.590931
12,"(Male, Midwest)",0.588824
41,"(Northeast, Old School)",0.584826
22,"(Old, New School)",0.575000
31,"(Old, Northeast)",0.573214
39,"(Midwest, Old School)",0.572683
38,"(Midwest, New School)",0.566190
35,"(Young, Northeast)",0.565372
34,"(Young, Midwest)",0.562862


In [183]:
qtreat = []
count = 0
test = ca.groupby(['region','school','gender','agegroup'])['place_percent'].mean()
test2 = ca.groupby(['region','school','gender','agegroup']).size()
for x in range(len(test)):
    qtreat.append([test.index[x],test[x],test2[x]])
    count += test2[x]
qtrt = pd.DataFrame(qtreat)
qtrt.rename(index=str, columns={0: "Treatment", 1: "Placement_Percentage", 2:"Count"},inplace=True)
qtrt.sort_values(by=['Count'],ascending=False,inplace=True)
#Placement_Percentage
qtrt

,Treatment,Placement_Percentage,Count
36,"(West, New School, Female, Young)",0.514931,48
45,"(West, Old School, Male, Young)",0.541048,47
42,"(West, Old School, Female, Young)",0.523697,47
39,"(West, New School, Male, Young)",0.575309,45
24,"(South, New School, Female, Young)",0.469648,41
27,"(South, New School, Male, Young)",0.529012,36
33,"(South, Old School, Male, Young)",0.484542,28
18,"(Northeast, Old School, Female, Young)",0.598333,25
30,"(South, Old School, Female, Young)",0.453344,24
12,"(Northeast, New School, Female, Young)",0.515509,24
